In [1]:
import pandas as pd
import re
import matplotlib.pyplot as plt
import numpy as np
import random
from collections import Counter
import itertools
import copy
import networkx as nx
import binascii

In [2]:
def split(word):
    return [char for char in word]

In [3]:
# Using readlines()
file1 = open('aoc-16', 'r')
file1 = open('aoc-16-ex', 'r')
Lines = file1.readlines()
print(Lines)

['8A004A801A8002F478620080001611562C8802118E34C0015000016115A2E0802F182340A0016C880162017C3686B18A3D4780']


In [4]:
conventions = {'0' : '0000',
'1' : '0001',
'2' : '0010',
'3' : '0011',
'4' : '0100',
'5' : '0101',
'6' : '0110',
'7' : '0111',
'8' : '1000',
'9' : '1001',
'A' : '1010',
'B' : '1011',
'C' : '1100',
'D' : '1101',
'E' : '1110',
'F' : '1111'}

typeConversion = {
    0 : '+',
    1 : '*',
    2 : 'min()',
    3 : 'max()',
    5 : 'grt()',
    6 : 'lt()',
    7 : '='
}

In [5]:
def covnertHexToBin(input):
    r = ''
    for i in range(0,len(input)):
        r += conventions[input[i]]
    return r

In [136]:
def convertBinToOjb(input):
    r = {   #package structure
        'version': None,
        'type': None,
        'lengthTypeId': None,
        'length': None,
        'sublength': None,
        'subCnt': None,
        'hdrLen': 0,
        'sub': None,
        'value': None,
        'data': [],
    }

    r['version'] = int(input[0:3],2)
    r['type'] = int(input[3:6],2)

    if r['type'] == 4:
        r['hdrLen'] = 6
        i = 6
        done = False
        while not done:
            done = (input[i] == '0')
            r['data'].append((input[i+1:i+5]))
            i += 5
        h = ''
        for ele in r['data']:
            h += str(ele)
        r['value'] = int(h,2)
        r['length'] = i #- 1 #protential Error works for now
    else:
        r['lengthTypeId'] = (input[6] == '0')
        if r['lengthTypeId']:
            r['hdrLen'] = 22
            r['length'] = int(input[7:22],2) + r['hdrLen']
            r['sublength'] = int(input[7:22],2)
            r['sub'] = input[22:]
            offset = 0
            while offset != r['sublength']:
                sub = convertBinToOjb(r['sub'][offset:r['sublength']+offset])
                offset += sub['length']
                r['data'].append(sub)
        else:
            r['hdrLen'] = 18
            r['length'] = r['hdrLen']
            r['subCnt'] = int(input[7:18],2)
            r['sub'] = input[18:]
            offset = 0
            for i in range(0,r['subCnt']):
                #print(i,'covnerting:',r['sub'][offset:])
                sub = convertBinToOjb(r['sub'][offset:])
                #print(i,'result: ',sub)
                offset += sub['length']
                #r['length'] += offset
                r['data'].append(sub)
                # print(sub['length'])

            for ele in r['data']:
                r['length'] += ele['length']

    return r

In [132]:
def resolve(r):
    #print(r)
    helper = r.copy()
    #for i in range(len(helper)-1,0,-1):
    for i in range(1,len(helper)):
        if type(helper[i]) is list:
            r[i] = resolve(helper[i])
            #print(r)

    if r[0] == '+':
        #print('sum',r)
        return sum(r[1:])
    if r[0] == '*':
        #print('prod',r)
        return np.prod(r[1:]) 
    if r[0] == 'min()':
        #print('min',min(r[1:]),r)
        return min(r[1:])
    if r[0] == 'max()':
        #print('max',max(r[1:]),r)
        return max(r[1:])        
    if r[0] == 'grt()':
        #print('grt()',r)
        return int(r[1] > r[2])
    if r[0] == 'lt()':
        #print('lt()',r)
        return int(r[1] < r[2])  
    if r[0] == '=':
        #print('==',r)
        return int(r[1] == r[2]) 
#
    return -1       

In [133]:
def getEQ(sub):
    r = []
    if sub['type'] != 4:
        r.append(typeConversion[sub['type']])
    for ele in sub['data']:
        if ele['type'] != 4:
            r.append(getEQ(ele))
        else:
            r.append(ele['value']) 

    return r      

In [150]:
#typeConversion = {
#    0 : '+',
#    1 : '*',
#    2 : 'min()',
#    3 : 'max()',
#    5 : 'grt()',
#    6 : 'lt()',
#    7 : '='

def calcTree(sub):

    r = 0

    for i,ele in enumerate(sub['data']):
        print(i,sub['data'])
        #sub['data'][i] = calcTree(ele)

    if sub['type'] == 0:
        for ele in sub['data']:
            r += ele['value']

    if sub['type'] == 1:
        r = 1
        for ele in sub['data']:
            r *= ele['value']

    if sub['type'] == 2:
        h = []
        for ele in sub['data']:
            h.append(ele['value'])
        r = min(h)

    if sub['type'] == 3:
        h = []
        for ele in sub['data']:
            h.append(ele['value'])
        r = max(h)
    if sub['type'] == 5:
        r = int(sub['data'][0]['value'] > sub['data'][1]['value'])
    if sub['type'] == 6:
        r = int(sub['data'][0]['value'] < sub['data'][1]['value'])
    if sub['type'] == 7: 
        r = int(sub['data'][0]['value'] == sub['data'][1]['value'])

    if sub['type'] == 4:
        return sub
    sub['value'] = r
    return sub 

In [134]:
def printSub(sub):
    if sub['type'] != 4:
        print('(',end='')
    if sub['type'] == 0:
        print('+',end='')
    if sub['type'] == 1:
        print('*',end='')
    if sub['type'] == 2:
        print('min',end='')
    if sub['type'] == 3:
        print('max',end='')
    if sub['type'] == 5:
        print('>',end='')        
    if sub['type'] == 6:
        print('<',end='')   
    if sub['type'] == 7:
        print('=',end='')        
    for ele in sub['data']:
        if ele['type'] != 4:
            printSub(ele)
        else:
            print(ele['value'],' ',end='')     
    if sub['type'] != 4:
        print(')',end='')                                

In [151]:
#print(getEQ(convertBinToOjb(covnertHexToBin('C200B40A82'))))
#print(getEQ(convertBinToOjb(covnertHexToBin('04005AC33890'))))
#print(getEQ(convertBinToOjb(covnertHexToBin('880086C3E88112'))))
#print(getEQ(convertBinToOjb(covnertHexToBin('CE00C43D881120'))))
#print(getEQ(convertBinToOjb(covnertHexToBin('D8005AC2A8F0'))))
#print(getEQ(convertBinToOjb(covnertHexToBin('F600BC2D8F'))))
#print(getEQ(convertBinToOjb(covnertHexToBin('9C005AC2F8F0'))))
#print(getEQ(convertBinToOjb(covnertHexToBin('9C0141080250320F1802104A08'))))
#print(resolve(getEQ(convertBinToOjb(covnertHexToBin('9C0141080250320F1802104A08')))))
#print(getEQ(convertBinToOjb(covnertHexToBin('9C0141080250320F1802104A08'))))
#print(resolve(getEQ(convertBinToOjb(covnertHexToBin('6051639005B56008C1D9BB3CC9DAD5BE97A4A9104700AE76E672DC95AAE91425EF6AD8BA5591C00F92073004AC0171007E0BC248BE0008645982B1CA680A7A0CC60096802723C94C265E5B9699E7E94D6070C016958F99AC015100760B45884600087C6E88B091C014959C83E740440209FC89C2896A50765A59CE299F3640D300827902547661964D2239180393AF92A8B28F4401BCC8ED52C01591D7E9D2591D7E9D273005A5D127C99802C095B044D5A19A73DC0E9C553004F000DE953588129E372008F2C0169FDB44FA6C9219803E00085C378891F00010E8FF1AE398803D1BE25C743005A6477801F59CC4FA1F3989F420C0149ED9CF006A000084C5386D1F4401F87310E313804D33B4095AFBED32ABF2CA28007DC9D3D713300524BCA940097CA8A4AF9F4C00F9B6D00088654867A7BC8BCA4829402F9D6895B2E4DF7E373189D9BE6BF86B200B7E3C68021331CD4AE6639A974232008E663C3FE00A4E0949124ED69087A848002749002151561F45B3007218C7A8FE600FC228D50B8C01097EEDD7001CF9DE5C0E62DEB089805330ED30CD3C0D3A3F367A40147E8023221F221531C9681100C717002100B36002A19809D15003900892601F950073630024805F400150D400A70028C00F5002C00252600698400A700326C0E44590039687B313BF669F35C9EF974396EF0A647533F2011B340151007637C46860200D43085712A7E4FE60086003E5234B5A56129C91FC93F1802F12EC01292BD754BCED27B92BD754BCED27B100264C4C40109D578CA600AC9AB5802B238E67495391D5CFC402E8B325C1E86F266F250B77ECC600BE006EE00085C7E8DF044001088E31420BCB08A003A72BF87D7A36C994CE76545030047801539F649BF4DEA52CBCA00B4EF3DE9B9CFEE379F14608')))))
# false == 176703 too low
#printSub(convertBinToOjb(covnertHexToBin('6051639005B56008C1D9BB3CC9DAD5BE97A4A9104700AE76E672DC95AAE91425EF6AD8BA5591C00F92073004AC0171007E0BC248BE0008645982B1CA680A7A0CC60096802723C94C265E5B9699E7E94D6070C016958F99AC015100760B45884600087C6E88B091C014959C83E740440209FC89C2896A50765A59CE299F3640D300827902547661964D2239180393AF92A8B28F4401BCC8ED52C01591D7E9D2591D7E9D273005A5D127C99802C095B044D5A19A73DC0E9C553004F000DE953588129E372008F2C0169FDB44FA6C9219803E00085C378891F00010E8FF1AE398803D1BE25C743005A6477801F59CC4FA1F3989F420C0149ED9CF006A000084C5386D1F4401F87310E313804D33B4095AFBED32ABF2CA28007DC9D3D713300524BCA940097CA8A4AF9F4C00F9B6D00088654867A7BC8BCA4829402F9D6895B2E4DF7E373189D9BE6BF86B200B7E3C68021331CD4AE6639A974232008E663C3FE00A4E0949124ED69087A848002749002151561F45B3007218C7A8FE600FC228D50B8C01097EEDD7001CF9DE5C0E62DEB089805330ED30CD3C0D3A3F367A40147E8023221F221531C9681100C717002100B36002A19809D15003900892601F950073630024805F400150D400A70028C00F5002C00252600698400A700326C0E44590039687B313BF669F35C9EF974396EF0A647533F2011B340151007637C46860200D43085712A7E4FE60086003E5234B5A56129C91FC93F1802F12EC01292BD754BCED27B92BD754BCED27B100264C4C40109D578CA600AC9AB5802B238E67495391D5CFC402E8B325C1E86F266F250B77ECC600BE006EE00085C7E8DF044001088E31420BCB08A003A72BF87D7A36C994CE76545030047801539F649BF4DEA52CBCA00B4EF3DE9B9CFEE379F14608')))
#convertBinToOjb(covnertHexToBin('9C0141080250320F1802104A08'))
#print(convertBinToOjb(covnertHexToBin('9C0141080250320F1802104A08')))
#printSub(convertBinToOjb(covnertHexToBin('9C0141080250320F1802104A08')))
#resolve(convertBinToOjb(covnertHexToBin('6051639005B56008C1D9BB3CC9DAD5BE97A4A9104700AE76E672DC95AAE91425EF6AD8BA5591C00F92073004AC0171007E0BC248BE0008645982B1CA680A7A0CC60096802723C94C265E5B9699E7E94D6070C016958F99AC015100760B45884600087C6E88B091C014959C83E740440209FC89C2896A50765A59CE299F3640D300827902547661964D2239180393AF92A8B28F4401BCC8ED52C01591D7E9D2591D7E9D273005A5D127C99802C095B044D5A19A73DC0E9C553004F000DE953588129E372008F2C0169FDB44FA6C9219803E00085C378891F00010E8FF1AE398803D1BE25C743005A6477801F59CC4FA1F3989F420C0149ED9CF006A000084C5386D1F4401F87310E313804D33B4095AFBED32ABF2CA28007DC9D3D713300524BCA940097CA8A4AF9F4C00F9B6D00088654867A7BC8BCA4829402F9D6895B2E4DF7E373189D9BE6BF86B200B7E3C68021331CD4AE6639A974232008E663C3FE00A4E0949124ED69087A848002749002151561F45B3007218C7A8FE600FC228D50B8C01097EEDD7001CF9DE5C0E62DEB089805330ED30CD3C0D3A3F367A40147E8023221F221531C9681100C717002100B36002A19809D15003900892601F950073630024805F400150D400A70028C00F5002C00252600698400A700326C0E44590039687B313BF669F35C9EF974396EF0A647533F2011B340151007637C46860200D43085712A7E4FE60086003E5234B5A56129C91FC93F1802F12EC01292BD754BCED27B92BD754BCED27B100264C4C40109D578CA600AC9AB5802B238E67495391D5CFC402E8B325C1E86F266F250B77ECC600BE006EE00085C7E8DF044001088E31420BCB08A003A72BF87D7A36C994CE76545030047801539F649BF4DEA52CBCA00B4EF3DE9B9CFEE379F14608')))

#print(getEQ(convertBinToOjb(covnertHexToBin('9C0141080250320F1802104A08'))))
#resolve(convertBinToOjb(covnertHexToBin('6051639005B56008C1D9BB3CC9DAD5BE97A4A9104700AE76E672DC95AAE91425EF6AD8BA5591C00F92073004AC0171007E0BC248BE0008645982B1CA680A7A0CC60096802723C94C265E5B9699E7E94D6070C016958F99AC015100760B45884600087C6E88B091C014959C83E740440209FC89C2896A50765A59CE299F3640D300827902547661964D2239180393AF92A8B28F4401BCC8ED52C01591D7E9D2591D7E9D273005A5D127C99802C095B044D5A19A73DC0E9C553004F000DE953588129E372008F2C0169FDB44FA6C9219803E00085C378891F00010E8FF1AE398803D1BE25C743005A6477801F59CC4FA1F3989F420C0149ED9CF006A000084C5386D1F4401F87310E313804D33B4095AFBED32ABF2CA28007DC9D3D713300524BCA940097CA8A4AF9F4C00F9B6D00088654867A7BC8BCA4829402F9D6895B2E4DF7E373189D9BE6BF86B200B7E3C68021331CD4AE6639A974232008E663C3FE00A4E0949124ED69087A848002749002151561F45B3007218C7A8FE600FC228D50B8C01097EEDD7001CF9DE5C0E62DEB089805330ED30CD3C0D3A3F367A40147E8023221F221531C9681100C717002100B36002A19809D15003900892601F950073630024805F400150D400A70028C00F5002C00252600698400A700326C0E44590039687B313BF669F35C9EF974396EF0A647533F2011B340151007637C46860200D43085712A7E4FE60086003E5234B5A56129C91FC93F1802F12EC01292BD754BCED27B92BD754BCED27B100264C4C40109D578CA600AC9AB5802B238E67495391D5CFC402E8B325C1E86F266F250B77ECC600BE006EE00085C7E8DF044001088E31420BCB08A003A72BF87D7A36C994CE76545030047801539F649BF4DEA52CBCA00B4EF3DE9B9CFEE379F14608')))
#print(resolve(getEQ(convertBinToOjb(covnertHexToBin('6051639005B56008C1D9BB3CC9DAD5BE97A4A9104700AE76E672DC95AAE91425EF6AD8BA5591C00F92073004AC0171007E0BC248BE0008645982B1CA680A7A0CC60096802723C94C265E5B9699E7E94D6070C016958F99AC015100760B45884600087C6E88B091C014959C83E740440209FC89C2896A50765A59CE299F3640D300827902547661964D2239180393AF92A8B28F4401BCC8ED52C01591D7E9D2591D7E9D273005A5D127C99802C095B044D5A19A73DC0E9C553004F000DE953588129E372008F2C0169FDB44FA6C9219803E00085C378891F00010E8FF1AE398803D1BE25C743005A6477801F59CC4FA1F3989F420C0149ED9CF006A000084C5386D1F4401F87310E313804D33B4095AFBED32ABF2CA28007DC9D3D713300524BCA940097CA8A4AF9F4C00F9B6D00088654867A7BC8BCA4829402F9D6895B2E4DF7E373189D9BE6BF86B200B7E3C68021331CD4AE6639A974232008E663C3FE00A4E0949124ED69087A848002749002151561F45B3007218C7A8FE600FC228D50B8C01097EEDD7001CF9DE5C0E62DEB089805330ED30CD3C0D3A3F367A40147E8023221F221531C9681100C717002100B36002A19809D15003900892601F950073630024805F400150D400A70028C00F5002C00252600698400A700326C0E44590039687B313BF669F35C9EF974396EF0A647533F2011B340151007637C46860200D43085712A7E4FE60086003E5234B5A56129C91FC93F1802F12EC01292BD754BCED27B92BD754BCED27B100264C4C40109D578CA600AC9AB5802B238E67495391D5CFC402E8B325C1E86F266F250B77ECC600BE006EE00085C7E8DF044001088E31420BCB08A003A72BF87D7A36C994CE76545030047801539F649BF4DEA52CBCA00B4EF3DE9B9CFEE379F14608')))))
#print(printSub(convertBinToOjb(covnertHexToBin('6051639005B56008C1D9BB3CC9DAD5BE97A4A9104700AE76E672DC95AAE91425EF6AD8BA5591C00F92073004AC0171007E0BC248BE0008645982B1CA680A7A0CC60096802723C94C265E5B9699E7E94D6070C016958F99AC015100760B45884600087C6E88B091C014959C83E740440209FC89C2896A50765A59CE299F3640D300827902547661964D2239180393AF92A8B28F4401BCC8ED52C01591D7E9D2591D7E9D273005A5D127C99802C095B044D5A19A73DC0E9C553004F000DE953588129E372008F2C0169FDB44FA6C9219803E00085C378891F00010E8FF1AE398803D1BE25C743005A6477801F59CC4FA1F3989F420C0149ED9CF006A000084C5386D1F4401F87310E313804D33B4095AFBED32ABF2CA28007DC9D3D713300524BCA940097CA8A4AF9F4C00F9B6D00088654867A7BC8BCA4829402F9D6895B2E4DF7E373189D9BE6BF86B200B7E3C68021331CD4AE6639A974232008E663C3FE00A4E0949124ED69087A848002749002151561F45B3007218C7A8FE600FC228D50B8C01097EEDD7001CF9DE5C0E62DEB089805330ED30CD3C0D3A3F367A40147E8023221F221531C9681100C717002100B36002A19809D15003900892601F950073630024805F400150D400A70028C00F5002C00252600698400A700326C0E44590039687B313BF669F35C9EF974396EF0A647533F2011B340151007637C46860200D43085712A7E4FE60086003E5234B5A56129C91FC93F1802F12EC01292BD754BCED27B92BD754BCED27B100264C4C40109D578CA600AC9AB5802B238E67495391D5CFC402E8B325C1E86F266F250B77ECC600BE006EE00085C7E8DF044001088E31420BCB08A003A72BF87D7A36C994CE76545030047801539F649BF4DEA52CBCA00B4EF3DE9B9CFEE379F14608'))))
#print(calcTree(convertBinToOjb(covnertHexToBin('6051639005B56008C1D9BB3CC9DAD5BE97A4A9104700AE76E672DC95AAE91425EF6AD8BA5591C00F92073004AC0171007E0BC248BE0008645982B1CA680A7A0CC60096802723C94C265E5B9699E7E94D6070C016958F99AC015100760B45884600087C6E88B091C014959C83E740440209FC89C2896A50765A59CE299F3640D300827902547661964D2239180393AF92A8B28F4401BCC8ED52C01591D7E9D2591D7E9D273005A5D127C99802C095B044D5A19A73DC0E9C553004F000DE953588129E372008F2C0169FDB44FA6C9219803E00085C378891F00010E8FF1AE398803D1BE25C743005A6477801F59CC4FA1F3989F420C0149ED9CF006A000084C5386D1F4401F87310E313804D33B4095AFBED32ABF2CA28007DC9D3D713300524BCA940097CA8A4AF9F4C00F9B6D00088654867A7BC8BCA4829402F9D6895B2E4DF7E373189D9BE6BF86B200B7E3C68021331CD4AE6639A974232008E663C3FE00A4E0949124ED69087A848002749002151561F45B3007218C7A8FE600FC228D50B8C01097EEDD7001CF9DE5C0E62DEB089805330ED30CD3C0D3A3F367A40147E8023221F221531C9681100C717002100B36002A19809D15003900892601F950073630024805F400150D400A70028C00F5002C00252600698400A700326C0E44590039687B313BF669F35C9EF974396EF0A647533F2011B340151007637C46860200D43085712A7E4FE60086003E5234B5A56129C91FC93F1802F12EC01292BD754BCED27B92BD754BCED27B100264C4C40109D578CA600AC9AB5802B238E67495391D5CFC402E8B325C1E86F266F250B77ECC600BE006EE00085C7E8DF044001088E31420BCB08A003A72BF87D7A36C994CE76545030047801539F649BF4DEA52CBCA00B4EF3DE9B9CFEE379F14608'))))
print(calcTree(convertBinToOjb(covnertHexToBin('9C0141080250320F1802104A08'))))


0 [{'version': 2, 'type': 0, 'lengthTypeId': False, 'length': 40, 'sublength': None, 'subCnt': 2, 'hdrLen': 18, 'sub': '01010000001100100000111100011000000000100001000001001010000010', 'value': None, 'data': [{'version': 2, 'type': 4, 'lengthTypeId': None, 'length': 11, 'sublength': None, 'subCnt': None, 'hdrLen': 6, 'sub': None, 'value': 1, 'data': ['0001']}, {'version': 4, 'type': 4, 'lengthTypeId': None, 'length': 11, 'sublength': None, 'subCnt': None, 'hdrLen': 6, 'sub': None, 'value': 3, 'data': ['0011']}]}, {'version': 6, 'type': 1, 'lengthTypeId': False, 'length': 40, 'sublength': None, 'subCnt': 2, 'hdrLen': 18, 'sub': '000100000100101000001000', 'value': None, 'data': [{'version': 0, 'type': 4, 'lengthTypeId': None, 'length': 11, 'sublength': None, 'subCnt': None, 'hdrLen': 6, 'sub': None, 'value': 2, 'data': ['0010']}, {'version': 2, 'type': 4, 'lengthTypeId': None, 'length': 11, 'sublength': None, 'subCnt': None, 'hdrLen': 6, 'sub': None, 'value': 2, 'data': ['0010']}]}]
1 [

In [121]:
convertBinToOjb(covnertHexToBin('6051639005B56008C1D9BB3CC9DAD5BE97A4A9104700AE76E672DC95AAE91425EF6AD8BA5591C00F92073004AC0171007E0BC248BE0008645982B1CA680A7A0CC60096802723C94C265E5B9699E7E94D6070C016958F99AC015100760B45884600087C6E88B091C014959C83E740440209FC89C2896A50765A59CE299F3640D300827902547661964D2239180393AF92A8B28F4401BCC8ED52C01591D7E9D2591D7E9D273005A5D127C99802C095B044D5A19A73DC0E9C553004F000DE953588129E372008F2C0169FDB44FA6C9219803E00085C378891F00010E8FF1AE398803D1BE25C743005A6477801F59CC4FA1F3989F420C0149ED9CF006A000084C5386D1F4401F87310E313804D33B4095AFBED32ABF2CA28007DC9D3D713300524BCA940097CA8A4AF9F4C00F9B6D00088654867A7BC8BCA4829402F9D6895B2E4DF7E373189D9BE6BF86B200B7E3C68021331CD4AE6639A974232008E663C3FE00A4E0949124ED69087A848002749002151561F45B3007218C7A8FE600FC228D50B8C01097EEDD7001CF9DE5C0E62DEB089805330ED30CD3C0D3A3F367A40147E8023221F221531C9681100C717002100B36002A19809D15003900892601F950073630024805F400150D400A70028C00F5002C00252600698400A700326C0E44590039687B313BF669F35C9EF974396EF0A647533F2011B340151007637C46860200D43085712A7E4FE60086003E5234B5A56129C91FC93F1802F12EC01292BD754BCED27B92BD754BCED27B100264C4C40109D578CA600AC9AB5802B238E67495391D5CFC402E8B325C1E86F266F250B77ECC600BE006EE00085C7E8DF044001088E31420BCB08A003A72BF87D7A36C994CE76545030047801539F649BF4DEA52CBCA00B4EF3DE9B9CFEE379F14608'))

{'version': 3,
 'type': 0,
 'lengthTypeId': True,
 'length': 5230,
 'sublength': 5208,
 'subCnt': None,
 'hdrLen': 22,
 'sub': '11100100000000010110110101011000000000100011000001110110011011101100111100110010011101101011010101101111101001011110100100101010010001000001000111000000001010111001110110111001100111001011011100100101011010101011101001000101000010010111101111011010101101100010111010010101011001000111000000000011111001001000000111001100000000010010101100000000010111000100000000011111100000101111000010010010001011111000000000000010000110010001011001100000101011000111001010011010000000101001111010000011001100011000000000100101101000000000100111001000111100100101001100001001100101111001011011100101101001100111100111111010010100110101100000011100001100000000010110100101011000111110011001101011000000000101010001000000000111011000001011010001011000100001000110000000000000100001111100011011101000100010110000100100011100000000010100100101011001110010000011111001110100000001000100000000